<a href="https://colab.research.google.com/github/marcellinus-witarsah/dogs-vs-cats-model/blob/main/dogs-vs-cats-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dogs vs Cats Model 
This project is based on competition from Kaggle which can be accessed using this link https://www.kaggle.com/competitions/dogs-vs-cats. This project aimed to create a model that can classify dogs and cats 

In [15]:
# import os
# import shutil
# import sys
# # code for making ensuring kaggle.json existed
# os.getcwd()
# kaggle_json_folder = os.path.join('/root/.kaggle')

# if not os.path.exists(kaggle_json_folder):
#   os.mkdir(path=kaggle_json_folder)
#   print(kaggle_json_folder + " has been created")
# else:
#   print(kaggle_json_folder + " already exists.")

# try:
#   source = os.path.join('/content/kaggle.json')
#   target = os.path.join(kaggle_json_folder, 'kaggle.json')
#   shutil.copy(source, target)
#   print("file copied")
# except IOError as e:
#     print("Unable to copy file. %s" % e)
# except:
#     print("Unexpected error:", sys.exc_info())


/root/.kaggle already exists.
file copied


## Data Preparation

Pull data from Kaggle using ***kaggle.json*** as its authentication token. Please make sure you have it inside ***/root/.kaggle/kaggle.json***

In [16]:
# download the data
# !pip install kaggle
# !kaggle competitions download -c dogs-vs-cats


 98% 795M/812M [00:03<00:00, 235MB/s]
100% 812M/812M [00:03<00:00, 239MB/s]


unzip the file and put it inside ***/dataset folder***. This extra folder it is just for preference just to make things look neat in the ***/content*** folder.

In [17]:
# # open the dogs-vs-cats.zip file
# from zipfile import ZipFile

# def extract_zip(filepath):
#     """
#     Function to open a extract zipped file
#     """
#     # opening the zip file in READ mode
#     with ZipFile(filepath, 'r') as zip:
#         #printing all the contents of the zip file
#         zip.printdir()
#         # extracting all the files
#         print('Extracting all the files now...')
#         zip.extractall(path=os.path.join(dataset_dir))
#         print('Done!')

# # specifying the zip file name
# file_name = "dogs-vs-cats.zip"

# dataset_dir = os.path.join("dataset")
# if not os.path.exists(dataset_dir):
#     os.mkdir(dataset_dir)
#     print(dataset_dir + " has been created")
    
# else:
#     print(dataset_dir + " already exists")

# if not os.path.exists(os.path.join(dataset_dir, file_name)):
#     shutil.move(file_name, dataset_dir)
# else:
#     print("data has already been moved")

# # opening the zip file in READ mode
# extract_zip(os.path.join(dataset_dir, file_name))
# extract_zip(os.path.join(dataset_dir, "train.zip"))
# extract_zip(os.path.join(dataset_dir, "test1.zip"))


Streaming output truncated to the last 5000 lines.
test1/550.jpg                                  2013-09-20 10:12:40        11253
test1/5500.jpg                                 2013-09-20 10:15:26        19488
test1/5501.jpg                                 2013-09-20 10:15:26        14965
test1/5502.jpg                                 2013-09-20 10:15:26        14406
test1/5503.jpg                                 2013-09-20 10:15:26        14957
test1/5504.jpg                                 2013-09-20 10:15:26        15065
test1/5505.jpg                                 2013-09-20 10:15:26         2985
test1/5506.jpg                                 2013-09-20 10:15:26         4349
test1/5507.jpg                                 2013-09-20 10:15:26        27502
test1/5508.jpg                                 2013-09-20 10:15:26        54253
test1/5509.jpg                                 2013-09-20 10:15:26         2312
test1/551.jpg                                  2013-09-20 10:12:40   